In [1]:
pwd

'/mnt/azmnt/code/Users/xai/tax-filer-prediction'

In [2]:
# Import workspace and other libraries

import os
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import Imputer
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from time import process_time

In [3]:
np.set_printoptions(suppress=True)

In [4]:
# Filepath
script_folder = os.getcwd()
filepath = script_folder + '/census-income-data.csv'
print(filepath + '\n')

/mnt/azmnt/code/Users/xai/tax-filer-prediction/census-income-data.csv



In [5]:
# Convert csv to datafreame
df = pd.read_csv(filepath)
df = df.sample(frac=1).reset_index(drop=True)
print("converted dataset to dataframe\n")

converted dataset to dataframe



In [6]:
## Start preprocessing: Clean up data and normalize

# Replace missing data
df_nan = df.replace('?', np.nan)
print("missing data replaced with NAN\n")

# trim all columns
df_clean = df_nan.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
print("trimmed all leading and trailing spaces\n")

print("preprocessing complete\n")

missing data replaced with NAN

trimmed all leading and trailing spaces

preprocessing complete



In [7]:
# Separate labels from measures
y_raw = df_clean.pop('Tax_Filer_Status')
#y_clean = y_raw.values.reshape((y_raw.shape[0],1))
X = df_clean
print("label separation complete\n")

label separation complete



In [8]:
# Drop columns with too many missing values (all migration-related columns)
X.drop(X.columns[[20]], axis = 1, inplace = True)
print("dropped instance weight\n")

dropped instance weight



**label encode discontinuous features**

In [10]:
x1_cont = X[['Age', 'Capital_Gains', 'Capital_Losses', 
             'Divdends_From_Stocks', 'Own_Business_Or_Self_Employed', 
             'Veterans_Benefits', 'Weeks_Worked_In_Year']].copy()
x1_disc = X.drop(['Age', 'Capital_Gains', 'Capital_Losses', 
             'Divdends_From_Stocks', 'Own_Business_Or_Self_Employed', 
             'Veterans_Benefits', 'Weeks_Worked_In_Year'], axis=1)

In [12]:
x1_cont_arr = x1_cont.to_numpy()
x1_disc_arr = x1_disc.to_numpy()

In [13]:
x1_cont_arr.shape, x1_disc_arr.shape

((196111, 7), (196111, 24))

In [54]:
# Encode discontinuous features
label_enc_features = preprocessing.LabelEncoder()

x1_disc_enc = np.zeros((x1_disc_arr.shape))
#encoded_vals = []
enc_dict = dict()

for i in range(x1_disc_arr.shape[1]):
    #print("Column {}: {}".format(i, x1_disc.columns[i]))
    column = x1_disc_arr[:, i]
    encoded = label_enc_features.fit_transform(column)
    #encoded_vals.append(encoded)
    x1_disc_enc[:, i] = encoded
    
    keys = label_enc_features.classes_
    values = label_enc_features.transform(label_enc_features.classes_)
    # elements = dict(zip(keys, values))
    elements = dict(zip(values, keys))
    dictionary = {x1_disc.columns[i]: elements}
    enc_dict.update(dictionary)

print("Encoded discrete features as labels")
#print(enc_dict)
#print(x1_disc_enc.shape)
                      
                       

Encoded discrete features as labels


In [55]:
for k, v in enc_dict.items():
    print((k, v), '\n')
    

('Class_Of_Worker', {0: 'Federal government', 1: 'Local government', 2: 'Never worked', 3: 'Private', 4: 'Self-employed-incorporated', 5: 'Self-employed-not incorporated', 6: 'State government', 7: 'Unknown', 8: 'Without pay'}) 

('Education', {0: '10th grade', 1: '11th grade', 2: '12th grade no diploma', 3: '1st 2nd 3rd or 4th grade', 4: '5th or 6th grade', 5: '7th and 8th grade', 6: '9th grade', 7: 'Associates degree-academic program', 8: 'Associates degree-occup /vocational', 9: 'Bachelors degree(BA AB BS)', 10: 'Doctorate degree(PhD EdD)', 11: 'High school graduate', 12: 'Less than 1st grade', 13: 'Masters degree(MA MS MEng MEd MSW MBA)', 14: 'Prof school degree (MD DDS DVM LLB JD)', 15: 'Some college but no degree'}) 

('Enrolled_In_Edu_Inst_Last_Wk', {0: 'College or university', 1: 'High school', 2: 'Unknown'}) 

('Marital_Status', {0: 'Divorced', 1: 'Married-A F spouse present', 2: 'Married-civilian spouse present', 3: 'Married-spouse absent', 4: 'Never married', 5: 'Separated',

**Completed encoding discontinuous features** #####

**Concatenate feature columns**

In [22]:
# Add the column headers back to the encoded feature array
x1_disc_df = pd.DataFrame(x1_disc_enc, columns=x1_disc.columns)
x1_disc_df

,Class_Of_Worker,Education,Enrolled_In_Edu_Inst_Last_Wk,Marital_Status,Major_Industry_Code,Major_Occupation_Code,Race,Hispanic_Origin,Sex,Member_Of_A_Labor_Union,...,Detailed_Household_And_Family_Stat,Detailed_Household_Summary_In_Household,Live_In_This_House_1_Year_Ago,Family_Members_Under_18,Country_Of_Birth_Father,Country_Of_Birth_Mother,Country_Of_Birth_Self,Citizenship,Fill_Inc_Questionnaire_For_Veteran's_Admin,Income_<>_$50000
0,3.0,1.0,1.0,4.0,18.0,11.0,4.0,7.0,0.0,1.0,...,8.0,2.0,0.0,2.0,39.0,39.0,39.0,4.0,1.0,0.0
1,3.0,15.0,0.0,4.0,18.0,11.0,4.0,7.0,0.0,1.0,...,8.0,2.0,1.0,0.0,39.0,39.0,39.0,4.0,1.0,0.0
2,3.0,9.0,2.0,2.0,10.0,7.0,4.0,7.0,1.0,1.0,...,8.0,2.0,2.0,1.0,39.0,39.0,39.0,4.0,1.0,0.0
3,7.0,15.0,2.0,2.0,21.0,14.0,4.0,7.0,0.0,1.0,...,20.0,4.0,0.0,4.0,39.0,39.0,39.0,4.0,1.0,0.0
4,7.0,5.0,2.0,6.0,21.0,14.0,4.0,7.0,0.0,1.0,...,37.0,7.0,1.0,4.0,39.0,39.0,39.0,4.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196106,7.0,0.0,1.0,4.0,21.0,14.0,4.0,7.0,1.0,1.0,...,8.0,2.0,1.0,0.0,25.0,25.0,39.0,4.0,1.0,0.0
196107,3.0,7.0,2.0,4.0,5.0,0.0,2.0,7.0,0.0,1.0,...,37.0,7.0,1.0,4.0,39.0,39.0,39.0,4.0,1.0,0.0
196108,6.0,15.0,0.0,4.0,5.0,0.0,2.0,7.0,0.0,1.0,...,8.0,2.0,1.0,2.0,39.0,39.0,39.0,4.0,1.0,0.0
196109,7.0,2.0,2.0,0.0,21.0,14.0,4.0,7.0,1.0,1.0,...,37.0,7.0,1.0,4.0,39.0,39.0,39.0,4.0,1.0,1.0


In [24]:
# Concatenate continuous and discrete (encoded) feature dataframes to compose the entire feature array
x_enc = pd.concat([x1_cont, x1_disc_df], axis=1)
x_enc

,Age,Capital_Gains,Capital_Losses,Divdends_From_Stocks,Own_Business_Or_Self_Employed,Veterans_Benefits,Weeks_Worked_In_Year,Class_Of_Worker,Education,Enrolled_In_Edu_Inst_Last_Wk,...,Detailed_Household_And_Family_Stat,Detailed_Household_Summary_In_Household,Live_In_This_House_1_Year_Ago,Family_Members_Under_18,Country_Of_Birth_Father,Country_Of_Birth_Mother,Country_Of_Birth_Self,Citizenship,Fill_Inc_Questionnaire_For_Veteran's_Admin,Income_<>_$50000
0,18,0,0,0,0,0,0,3.0,1.0,1.0,...,8.0,2.0,0.0,2.0,39.0,39.0,39.0,4.0,1.0,0.0
1,20,0,0,0,0,0,0,3.0,15.0,0.0,...,8.0,2.0,1.0,0.0,39.0,39.0,39.0,4.0,1.0,0.0
2,46,0,0,1,0,2,0,3.0,9.0,2.0,...,8.0,2.0,2.0,1.0,39.0,39.0,39.0,4.0,1.0,0.0
3,37,0,0,0,0,2,52,7.0,15.0,2.0,...,20.0,4.0,0.0,4.0,39.0,39.0,39.0,4.0,1.0,0.0
4,58,0,0,0,0,2,52,7.0,5.0,2.0,...,37.0,7.0,1.0,4.0,39.0,39.0,39.0,4.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196106,17,0,0,0,0,0,0,7.0,0.0,1.0,...,8.0,2.0,1.0,0.0,25.0,25.0,39.0,4.0,1.0,0.0
196107,31,0,0,0,0,2,0,3.0,7.0,2.0,...,37.0,7.0,1.0,4.0,39.0,39.0,39.0,4.0,1.0,0.0
196108,22,0,0,1500,0,0,0,6.0,15.0,0.0,...,8.0,2.0,1.0,2.0,39.0,39.0,39.0,4.0,1.0,0.0
196109,61,0,0,0,2,2,52,7.0,2.0,2.0,...,37.0,7.0,1.0,4.0,39.0,39.0,39.0,4.0,1.0,1.0


**Encode labels**

In [27]:
# Encode labels
label_enc = preprocessing.LabelEncoder()
Y = label_enc.fit_transform(y_raw)

In [28]:
np.unique(label_enc.inverse_transform(Y))

array(['Nonfiler', 'Tax Filer or N/A'], dtype=object)

**Define features and labels for visualizing the explanation later**

In [32]:
feature_names = np.array(x_enc.columns.values)
print("Features have been defined\n")
#feature_names

Features have been defined



In [33]:
labels = np.array(['tax filer', 'non-filer']).tolist()
print("classes have been defined\n")

classes have been defined



**Start training**

In [34]:
# Convert feature set to numpy array
x_arr = x_enc.to_numpy() 

In [35]:
X_train,X_test,y_train,y_test = train_test_split(x_arr, Y, test_size=0.3)
print("split dataset into training and test sets with an 70-30 partition\n")

split dataset into training and test sets with an 70-30 partition



In [36]:
# Flatten label column to fit the SVM classifier requirement (otherwise you get warnings)
y_train_flat = y_train.ravel()
y_test_flat = y_test.ravel()
print("flattened class label array\n")

flattened class label array



In [37]:
# Impute missing values using univariate imputer (multivariate is not supported in this version of Scikit-learn 0.18.1)
print("imputing missing values as nan\n")
imp = Imputer(missing_values=np.nan, strategy='most_frequent')
imp.fit(X_train)
X_train_clean = imp.transform(X_train)
X_test_clean = imp.transform(X_test)

print("imputation complete\n")

imputing missing values as nan



/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


imputation complete



In [ ]:
# Normalize dataset (exclude labels)
#X_norm = normalize(X_clean)

# Standardize dataset
scaler = StandardScaler()  

# Fit only on training data
scaler.fit(X_train_clean[:,1:])
x_train_std = scaler.transform(X_train_clean[:, 1:])
train = np.concatenate(( X_train_clean[:,0], X_train_clean[:,1:]), axis=0)

# apply same transformation to test data
x_test = scaler.transform(X_test_clean[:, 1:], axis=0)
test = np.concatenate(( X_test_clean[:,0], X_test_clean[:,1:]), axis=1)

print("standardized dataset\n")

## Train and Test

In [38]:
# Train a shallow neural network
clf = MLPClassifier(solver='sgd',
                    hidden_layer_sizes=(15,30,15), random_state=1)

start = process_time()
model = clf.fit(X_train_clean, y_train_flat)
end = process_time()
print("Training complete, took {:.0f} seconds".format(np.round(end-start)))

**Decode feature labels**

In [56]:
for k, v in enc_dict.items():
    print((k.strip(), v), '\n')

('Class_Of_Worker', {0: 'Federal government', 1: 'Local government', 2: 'Never worked', 3: 'Private', 4: 'Self-employed-incorporated', 5: 'Self-employed-not incorporated', 6: 'State government', 7: 'Unknown', 8: 'Without pay'}) 

('Education', {0: '10th grade', 1: '11th grade', 2: '12th grade no diploma', 3: '1st 2nd 3rd or 4th grade', 4: '5th or 6th grade', 5: '7th and 8th grade', 6: '9th grade', 7: 'Associates degree-academic program', 8: 'Associates degree-occup /vocational', 9: 'Bachelors degree(BA AB BS)', 10: 'Doctorate degree(PhD EdD)', 11: 'High school graduate', 12: 'Less than 1st grade', 13: 'Masters degree(MA MS MEng MEd MSW MBA)', 14: 'Prof school degree (MD DDS DVM LLB JD)', 15: 'Some college but no degree'}) 

('Enrolled_In_Edu_Inst_Last_Wk', {0: 'College or university', 1: 'High school', 2: 'Unknown'}) 

('Marital_Status', {0: 'Divorced', 1: 'Married-A F spouse present', 2: 'Married-civilian spouse present', 3: 'Married-spouse absent', 4: 'Never married', 5: 'Separated',

In [74]:
for k, v in enc_dict['Class_Of_Worker'].items():
    print(k, v)

0 Federal government
1 Local government
2 Never worked
3 Private
4 Self-employed-incorporated
5 Self-employed-not incorporated
6 State government
7 Unknown
8 Without pay


In [91]:
x1_disc_arr[:,0]

array(['Private', 'Private', 'Private', ..., 'State government',
       'Unknown', 'Private'], dtype=object)

In [90]:
x1_disc_enc[:,0]

array([3., 3., 3., ..., 6., 7., 3.])

In [95]:
[enc_dict['Class_Of_Worker'].get(items, items) for items in x1_disc_arr[:, 0]]

['Private',
 'Private',
 'Private',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Self-employed-not incorporated',
 'Local government',
 'Private',
 'Private',
 'Private',
 'Private',
 'Private',
 'State government',
 'Local government',
 'Private',
 'Local government',
 'Private',
 'Private',
 'Self-employed-not incorporated',
 'Local government',
 'Unknown',
 'Private',
 'Unknown',
 'Private',
 'Local government',
 'Private',
 'Private',
 'Private',
 'Private',
 'Private',
 'Private',
 'Private',
 'Unknown',
 'Unknown',
 'Private',
 'Private',
 'Private',
 'Private',
 'Unknown',
 'Private',
 'Private',
 'Self-employed-not incorporated',
 'Private',
 'Local government',
 'Unknown',
 'Unknown',
 'State government',
 'Self-employed-not incorporated',
 'Local government',
 'Unknown',
 'Private',
 'Self-employed-not incorporated',
 'Private',
 'Federal government',
 'Unknown',
 'Private',
 'Private',
 'Unknown',
 'Private',
 'Private',
 'Private',
 'Private',
 'Unknown',
 'Private',
 'Private',
 'Private',
 'Unknown',
 'Private',
 'Unknown',
 'Private',
 'Private',
 'Unknown',
 'Unknown',
 'Private',
 'Unknown',
 'Private',
 'Private',
 'Unknown',
 'Private',
 'Unknown',
 'Private',
 'Private',
 'Private',
 'Self-employed-not incorporated',
 'Unknown',
 'Self-employed-not incorporated',
 'Local government',
 'Private',
 'Private',
 'Private',
 'Local government',
 'Unknown',
 'Private',
 'Private',
 'Private',
 'Unknown',
 'Unknown',
 'Private',
 'Private',
 'Private',
 'Self-employed-not incorporated',
 'Private',
 'Self-employed-not incorporated',
 'Private',
 'Self-employed-not incorporated',
 'Private',
 'Unknown',
 'Private',
 'Local government',
 'Private',
 'Private',
 'Private',
 'Private',
 'Private',
 'Private',
 'Unknown',
 'Private',
 'Private',
 'Private',
 'Private',
 'Private',
 'Private',
 'Unknown',
 'Self-employed-incorporated',
 'Private',
 'Private',
 'Private',
 'Private',
 'Private',
 'Private',
 'Unknown',
 'Private',
 'Unknown',
 'Unknown',
 'Private',
 'Private',
 'Unknown',
 'Unknown',
 'Private',
 'Self-employed-not incorporated',
 'Local government',
 'Federal government',
 'Private',
 'Unknown',
 'Unknown',
 'Private',
 'Unknown',
 'Self-employed-not incorporated',
 'Unknown',
 'Self-employed-not incorporated',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Private',
 'Unknown',
 'State government',
 'Private',
 'Private',
 'Private',
 'Self-employed-incorporated',
 'Local government',
 'Unknown',
 'State government',
 'Unknown',
 'Private',
 'Private',
 'Private',
 'Unknown',
 'Private',
 'Unknown',
 'Private',
 'Private',
 'Private',
 'Private',
 'Local government',
 'Private',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Private',
 'Private',
 'Self-employed-not incorporated',
 'Private',
 'Private',
 'Unknown',
 'Unknown',
 'Unknown',
 'Private',
 'Private',
 'State government',
 'Private',
 'Unknown',
 'Private',
 'Unknown',
 'Private',
 'Unknown',
 'Private',
 'Private',
 'Private',
 'Private',
 'Private',
 'Unknown',
 'Unknown',
 'Private',
 'Private',
 'Unknown',
 'Unknown',
 'Private',
 'Private',
 'Unknown',
 'Local government',
 'Private',
 'Unknown',
 'Unknown',
 'Local government',
 'Self-employed-incorporated',
 'Unknown',
 'Unknown',
 'Self-employed-not incorporated',
 'Unknown',
 'Unknown',
 'Federal government',
 'Private',
 'Unknown',
 'Unknown',
 'Unknown',
 'Private',
 'Private',
 'Private',
 'Local government',
 'Unknown',
 'Private',
 'Unknown',
 'Private',
 'Unknown',
 'Private',
 'Unknown',
 'State government',
 'Local government',
 'Unknown',
 'Private',
 'Private',
 'Private',
 'Unknown',
 'Private',
 'Unknown',
 'Private',
 'Private',
 'Unknown',
 'Private',
 'Private',
 'Private',
 'Unknown',
 'Private',
 'Unknown',
 'Private',
 'Private',
 'Unknown',
 'Private',
 'Unknown',
 'Private',
 'Unknown',
 'Private',
 'Private',
 'Local government',
 'Private',
 'Private',
 'Private',
 'Private',
 'Self-employed-not incorporated',
 'Private',
 'Private',
 '

In [93]:
for keys in enc_dict.keys():
    print( keys)

Class_Of_Worker
Education
Enrolled_In_Edu_Inst_Last_Wk
Marital_Status
Major_Industry_Code
Major_Occupation_Code
Race
Hispanic_Origin
Sex
Member_Of_A_Labor_Union
Reason_For_Unemployment
Full_Or_Part_Time_Employment_Stat
Region_Of_Previous_Residence
State_Of_Previous_Residence
Detailed_Household_And_Family_Stat
Detailed_Household_Summary_In_Household
Live_In_This_House_1_Year_Ago
Family_Members_Under_18
Country_Of_Birth_Father
Country_Of_Birth_Mother
Country_Of_Birth_Self
Citizenship
Fill_Inc_Questionnaire_For_Veteran's_Admin
Income_<>_$50000


In [122]:
X_train_clean.shape[0], x1_cont_arr.shape[1]

(137277, 7)

In [141]:
continous_train = np.zeros( (X_train_clean.shape[0], x1_cont_arr.shape[1]) )
discontinous_train =  np.char.asarray( np.zeros( (X_train_clean.shape[0], x1_disc_arr.shape[1]) ), itemsize=40, unicode=True )

continous_train.shape, discontinous_train.shape

((137277, 7), (137277, 24))

In [176]:
# Decode discontinuous features

#rev_enc_dict = [ v:k for k, v in enc_dict.iteritems() ]

def decode_features(ndarray_row, ndarray_col_cont, ndarray_col_disc, dictionary):
    
	continous_train = np.zeros( (ndarray_row.shape[0], ndarray_col_cont.shape[1]) )
	discontinous_train =  np.char.asarray( np.zeros( (ndarray_row.shape[0], ndarray_col_disc.shape[1]) ), 
                                          itemsize=40, unicode=True )

	decoded = np.concatenate( (continous_train, discontinous_train), axis=1)

	i = 0
	for i in range(continous_train.shape[1]):
		column = ndarray_row[:, i]
		decoded[:, i] = column
		i +=1

	i = 7
	for key in dictionary.keys():
		column = [ dictionary[key].get(items, items) for items in ndarray_row[:, i] ]
		decoded[:, i] = column
		i +=1

	return decoded

In [183]:
x1_dec_train = decode_features(X_train_clean, x1_cont_arr, x1_disc_arr, enc_dict)
x1_dec_train.shape

(137277, 31)

In [178]:
x1_dec_test = decode_features(X_test_clean, x1_cont_arr, x1_disc_arr, enc_dict)
x1_dec_test.shape

(58834, 31)

In [169]:
x1_cont.columns[0:]

Index(['Age', 'Capital_Gains', 'Capital_Losses', 'Divdends_From_Stocks',
       'Own_Business_Or_Self_Employed', 'Veterans_Benefits',
       'Weeks_Worked_In_Year'],
      dtype='object')

In [172]:
x1_disc.columns[0:6]

Index(['Class_Of_Worker', 'Education', 'Enrolled_In_Edu_Inst_Last_Wk',
       'Marital_Status', 'Major_Industry_Code', 'Major_Occupation_Code'],
      dtype='object')

In [188]:
x1_dec_train[:,0]

array(['57.0', '36.0', '52.0', ..., '37.0', '26.0', '38.0'], dtype='<U40')

In [ ]:
len(X_test_clean)

In [39]:
predictions = model.predict(x1_dec_test)

In [40]:
print(confusion_matrix(y_test_flat,predictions))

[[18278  3227]
 [  372 36957]]


In [41]:
print(classification_report(y_test_flat,predictions))

              precision    recall  f1-score   support

           0       0.98      0.85      0.91     21505
           1       0.92      0.99      0.95     37329

   micro avg       0.94      0.94      0.94     58834
   macro avg       0.95      0.92      0.93     58834
weighted avg       0.94      0.94      0.94     58834



In [ ]:
# De-standardize data
x_train_dstd = scaler.inverse_transform(x_train)
x_test_dstd = scaler.inverse_transform(x_test)

**Some unit testing to ensure things are working correctly**

In [ ]:
np.max(df['Age'])

In [ ]:
np.min(x_test[0])

In [ ]:
np.max(x_test_dstd[0])

In [ ]:
y_test_flat[220:231]

In [ ]:
x_test[0:10,0]

In [ ]:
x_test_dstd[0:10,0].astype(int)

**Outputs are reversible. Unit testing complete.**

## Start visualization

In [200]:
from interpret.ext.blackbox import TabularExplainer

explainer = TabularExplainer(model, 
                            X_test_clean, # x_test
                            features=feature_names, 
                            classes=labels)
print("initialized explainer\n")

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/linear.py:49: UserWarning: The default value for feature_dependence has been changed to "independent"!
  warnings.warn('The default value for feature_dependence has been changed to "independent"!')


initialized explainer



In [201]:
# you can use the training data or the test data here
global_explanation = explainer.explain_global(X_test_clean[0:100])

# if you used the PFIExplainer in the previous step, use the next line of code instead
# global_explanation = explainer.explain_global(x_train, true_labels=y_test)

# sorted feature importance values and feature names
sorted_global_importance_values = global_explanation.get_ranked_global_values()
sorted_global_importance_names = global_explanation.get_ranked_global_names()
dict(zip(sorted_global_importance_names, sorted_global_importance_values))

# alternatively, you can print out a dictionary that holds the top K feature names and values
global_explanation.get_feature_importance_dict()

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \


{'Divdends_From_Stocks': 0.11944825718019078,
 'Age': 0.10622315755349548,
 'Capital_Gains': 0.04259878438697495,
 'Country_Of_Birth_Self': 0.010970093693264779,
 'State_Of_Previous_Residence': 0.010863951145400135,
 'Weeks_Worked_In_Year': 0.0033788725162111587,
 'Education': 0.002197333409739675,
 'Major_Industry_Code': 0.001991596325022829,
 'Detailed_Household_Summary_In_Household': 0.0019078433914876236,
 "Fill_Inc_Questionnaire_For_Veteran's_Admin": 0.0012346552928826273,
 'Citizenship': 0.0011692190737850913,
 'Hispanic_Origin': 0.0011400772994591703,
 'Member_Of_A_Labor_Union': 0.0006005316116808627,
 'Marital_Status': 0.0005777997523246477,
 'Family_Members_Under_18': 0.0004684610528846012,
 'Detailed_Household_And_Family_Stat': 0.0004295552776892741,
 'Full_Or_Part_Time_Employment_Stat': 0.00039709702324442354,
 'Veterans_Benefits': 0.0003796783405110112,
 'Reason_For_Unemployment': 0.000348213099582622,
 'Live_In_This_House_1_Year_Ago': 0.00025353027027927316,
 'Enrolled_In_

In [220]:
global_explanation.local_importance_values

[[[0.1063513428373075,
   0.04104811248293695,
   0.0,
   0.15297741080951782,
   0.0,
   0.0,
   -0.005279644647696535,
   0.0,
   -0.0035132785136629124,
   0.0,
   0.0,
   -0.00589400181138023,
   0.0,
   0.0,
   -0.002322874812045947,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -0.014040268662352098,
   0.0,
   -0.006842427271013002,
   -0.003122907306140038,
   0.0,
   0.0,
   0.0,
   -0.016572423533606134,
   0.0,
   -0.004204300026973112,
   0.0],
  [0.10863388505610297,
   0.033326413391483525,
   0.0,
   0.15110088671693767,
   0.0,
   0.0,
   -0.007036516371420352,
   0.0,
   0.0,
   0.0,
   -0.0036757400857649405,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -0.0036586776119095915,
   0.0,
   0.0,
   -0.009488713928862691,
   0.0,
   0.0,
   -0.004505870514584363,
   0.0,
   0.0,
   0.0,
   -0.012359193042183358,
   -0.004644351369210542,
   -0.004260233764163179,
   -0.004847148931532985],
  [0.10714296417868917,
   0.034253456051567543,
   0.0,
   0.15375366361257084,
   0.0,
   0.0,
   -0.0075113101041457825,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -0.006680185363619751,
   0.0,
   0.0,
   0.0,
   -0.017161685185894504,
   0.0,
   -0.007711751221711416,
   0.0,
   0.0,
   0.0,
   0.0,
   -0.017500412422563894,
   0.0,
   0.0,
   0.0],
  [0.10070710951553563,
   0.040543501640923935,
   0.0,
   0.15513783710097215,
   0.0,
   0.0,
   -0.008914685329931366,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -0.007098867616471716,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -0.01487019494248204,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -0.006982470509902555,
   -0.011399875264450993,
   0.0,
   -0.008537615049300812,
   0.0],
  [0.10471648776524761,
   0.04524128384905107,
   0.0,
   0.1544403998904397,
   0.0,
   0.0,
   -0.0037119960265912383,
   0.0,
   -0.00839733780572064,
   0.0,
   0.0,
   -0.005641451154600963,
   -0.0029477914441785047,
   0.0,
   0.0,
   -0.0035274741551486744,
   -0.006047428358083794,
   0.0,
   0.005176631974232751,
   -0.0026693662709952395,
   -0.020123741905818244,
   0.0,
   -0.00884668523749016,
   0.0,
   0.0,
   0.0,
   0.0,
   -0.007094830679210629,
   -0.001981960896240753,
   0.0,
   0.0],
  [0.10597609857085863,
   0.04928910617165051,
   0.0,
   0.1448131896052975,
   0.0,
   0.0,
   -0.0045951605564655135,
   0.0,
   -0.003980392521399735,
   0.0,
   -0.004038239177131468,
   -0.00524542507265123,
   0.0,
   0.0,
   -0.007975992150296957,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -0.02028569505893213,
   0.0,
   -0.005096771919383486,
   0.0,
   0.0,
   0.0,
   0.0,
   -0.0057426044528688935,
   0.0,
   -0.004533373893784914,
   0.0],
  [0.10369766009072168,
   0.05102811038875343,
   0.0,
   0.12907307609929897,
   0.0,
   0.0,
   -0.004553231898845328,
   0.0,
   -0.0053241426952259615,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -0.004343215354498751,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -0.0147072715362391,
   0.0,
   -0.005262435233257017,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -0.005282800357535727,
   -0.0057410099582799745,
   0.0],
  [0.10456016489892969,
   0.04738232021403275,
   0.0,
   0.12063321428583385,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -0.007927742899306367,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -0.018405651861035183,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -0.007657565093562496,
   0.0],
  [0.10164307097882777,
   0.04628920782456597,
   0.0,
   0.11649957190438187,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -0.01870427765495422,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -0.007142833507929186,
   0.0,
   0.0],
  [0.09929197948452607,
   0.05397043292773898,
   0.0,
   0.1154090423909647,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
  

In [221]:
# explain the first data point in the test set
local_explanation = explainer.explain_local(X_test_clean[0:100])

# sorted feature importance values and feature names
sorted_local_importance_names = local_explanation.get_ranked_local_names()
sorted_local_importance_values = local_explanation.get_ranked_local_values()

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/anaconda/en

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/anaconda/en

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/anaconda/en

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/anaconda/en

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/anaconda/en

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!

/anaconda/en

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/shap/explainers/kernel.py:545: UserWarning:

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!



In [224]:
local_explanation.

KeyError: 'scores'

In [225]:
from azureml.contrib.interpret.visualize import ExplanationDashboard

ExplanationDashboard(global_explanation, model, X_test_clean[0:100])

ExplanationWidget(value={'predictedY': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…

In [ ]:
# save model

import pickle
model_name = 'xai_poc_model_1.pkl'
pickle.dump(model_1, open(model_name, 'wb'))

In [ ]:
# load the model from disk
filename = 'xai_poc_model.pkl'
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
# load the model from disk
filename1 = 'xai_poc_model_1.pkl'
loaded_model_1 = pickle.load(open(filename1, 'rb'))